In [ ]:
from openpyxl import load_workbook
import PharmOps.io as pharmio
import pandas as pd
import matplotlib.pyplot as plt
filepathDAR = r'E:\pharmacodynamics\sample data\summary table\DAR Ki values 34485-494.xlsx'
filepath5HT = r'E:\pharmacodynamics\sample data\summary table\5HTR KI table 34485-34494.xlsx'
wbDAR = load_workbook(filepathDAR, data_only=True)
wb5HT = load_workbook(filepath5HT, data_only=True)

In [ ]:
summaryDAR = pharmio.load_binding_summary_excel(filepathDAR)
summary5HT = pharmio.load_binding_summary_excel(filepath5HT)

In [ ]:
print(summaryDAR)
print(summary5HT)
combined = {}
for key in summaryDAR:
    if (key in summary5HT and 
            isinstance(summaryDAR[key], dict) and 
            isinstance(summary5HT[key], dict)):
        combined[key] = {**summaryDAR[key], **summary5HT[key]}
        
print(combined)

In [ ]:
def round_sig(x, sigFigs):
    rounded = '{:g}'.format(float('{:.{p}g}'.format(x, p=sigFigs)))
    if len(rounded) < sigFigs and '.' not in rounded:
        rounded = rounded + '.'
    while len(rounded) <= sigFigs and '.' in rounded:
        rounded = rounded + '0'
    return rounded

meanPrecision = 3
semPrecision = 2
for key, nestedDict in combined.items():
    print(f"Drug: {key}")
    df = pd.DataFrame()
    df['Receptor'] = list(combined[key].keys())
    ic50 = []
    ki = []
    hillSlope = []
    for subKey, values in nestedDict.items():
        if isinstance(values, dict):
            if 'mean' in values and 'sem' in values:
                meanVals = values['mean']
                semVals = values['sem']
                combinedVals = {}
                for meanKey, semKey in zip(meanVals.keys(), semVals.keys()):
                    outputStr = f"{round_sig(meanVals[meanKey], meanPrecision)} ± {round_sig(semVals[semKey], semPrecision)}"
                    match(meanKey):
                        case('ic50'):
                            ic50.append(outputStr)
                        case('ki'):
                            ki.append(outputStr)
                        case('hillSlope'):
                            hillSlope.append(outputStr)
    df['IC₅₀ (nM) ± SEM'] = ic50
    df['Ki (nM) ± SEM'] = ki
    df['Hill Slope ± SEM'] = hillSlope
    fig, ax = plt.subplots()
    ax.axis('tight')
    ax.axis('off')
    ax.table(cellText=df.values,
            colLabels=df.columns,
            loc='center',
            cellLoc='center')
    plt.tight_layout()
    plt.savefig(f'..\sample data\{key}.png', bbox_inches='tight')

In [ ]:
import PharmOps.io as pharmio
pharmio.make_function_table(r'E:\pharmacodynamics\sample data\summary table')
